In [1]:
import os
import random
import numpy as np
import pandas as pd
import tensorflow as tf

from tqdm.notebook import tqdm
from nasbench import api
from search_spaces import load_nasbench_101
from random_search import run_random_search, random_spec
from neural_predictor import classifier, regressor, regressor_mlp, regressor_cnn
from input_preprocessing import preprocess_nasbench

In [2]:
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "1"
os.environ["TF_FORCE_GPU_ALLOW_GROWTH"] = "true"

In [3]:
nasbench = load_nasbench_101()

Loading dataset from file... This may take a few minutes...
Instructions for updating:
Use eager execution and: 
`tf.data.TFRecordDataset(path)`
Loaded dataset in 44 seconds


In [4]:
def get_N_samples(N):
    models = []

    for _ in range(N):
        while True:
            model = random_spec(nasbench)
            if model not in models:
                models.append(nasbench.query(model))
                break
                
    return preprocess_nasbench(models)

### Ablation Study 1: N vs K and Ablation Study

In [5]:
LOOPS = 60
MAX_SAMPLES = 5000
MAX_TIME_BUDGET = 8e5

In [6]:
results = {}

In [ ]:
for N in [43, 86, 129, 172, 344, 860]:
    train_data = get_N_samples(N)
    reg = regressor([train_data['X'], train_data['norm_A'], train_data['norm_AT']], train_data['val_acc'])
    train_time = np.sum(train_data['times'])
    
    np_val_avg, np_test_avg = [], []
    np_val_std, np_test_std = [], []

    val_acc, test_acc = [], []
    for budget in tqdm(range(int(train_time), int(MAX_TIME_BUDGET), 1600)): # 500 loops
        time_spent = 0
        test_models = get_N_samples(N+budget//100)

        for i in range(len(test_models['times'])):
            time_spent = time_spent + test_models['times'][i]
            if time_spent >= budget:
                break

        X = test_models['X'][:i]
        A = test_models['norm_A'][:i]
        AT = test_models['norm_AT'][:i]
        labels = test_models['val_acc'][:i]
        test_labels = test_models['test_acc'][:i]

        pred_acc = reg.predict([X, A, AT]).ravel()

        selected_val = labels
        selected_test = test_labels

        best_val_idx = np.argmax(selected_val)

        val_acc.append(selected_val[best_val_idx])
        test_acc.append(selected_test[best_val_idx])

        np_val_avg.append(np.max(val_acc))
        np_val_std.append(np.std(val_acc))

        np_test_avg.append(np.max(test_acc))
        np_test_std.append(np.std(test_acc))

    results[N] = {
        'np_val_avg': np_val_avg,
        'np_test_avg': np_test_avg,
        'np_val_std': np_val_std,
        'np_test_std': np_test_std
    }

In [15]:
np.save('outputs/ablation_study_by_time.npy', results)

### Ablation Study 2: MLP vs CNN vs GCN

In [6]:
# MLP
N = 172
train_data = get_N_samples(N)
reg = regressor_mlp(np.concatenate([train_data['X'], train_data['norm_A']], axis=-1), train_data['val_acc'])
train_time = np.sum(train_data['times'])

np_val_avg, np_test_avg = [], []
np_val_std, np_test_std = [], []

val_acc, test_acc = [], []
for budget in tqdm(range(int(train_time), int(MAX_TIME_BUDGET), 1600)): # 500 loops
    time_spent = 0
    test_models = get_N_samples(N+budget//100)

    for i in range(len(test_models['times'])):
        time_spent = time_spent + test_models['times'][i]
        if time_spent >= budget:
            break

    X = test_models['X'][:i]
    A = test_models['norm_A'][:i]
    AT = test_models['norm_AT'][:i]
    labels = test_models['val_acc'][:i]
    test_labels = test_models['test_acc'][:i]

    pred_acc = reg.predict(np.concatenate([X, A], axis=-1)).ravel()

    selected_val = labels
    selected_test = test_labels

    best_val_idx = np.argmax(selected_val)

    val_acc.append(selected_val[best_val_idx])
    test_acc.append(selected_test[best_val_idx])

    np_val_avg.append(np.max(val_acc))
    np_val_std.append(np.std(val_acc))

    np_test_avg.append(np.max(test_acc))
    np_test_std.append(np.std(test_acc))

  0%|          | 0/367 [00:00<?, ?it/s]

In [7]:
result_mlp = {
    'np_val_avg': np_val_avg,
    'np_test_avg': np_test_avg,
    'np_val_std': np_val_std,
    'np_test_std': np_test_std
}

In [8]:
np.save('outputs/ablation_study_by_arch_time_mlp.npy', result_mlp)

In [9]:
# CNN
N = 172
train_data = get_N_samples(N)
reg = regressor_cnn(np.concatenate([train_data['X'], train_data['norm_A']], axis=-1), train_data['val_acc'])
train_time = np.sum(train_data['times'])

np_val_avg, np_test_avg = [], []
np_val_std, np_test_std = [], []

val_acc, test_acc = [], []
for budget in tqdm(range(int(train_time), int(MAX_TIME_BUDGET), 1600)): # 500 loops
    time_spent = 0
    test_models = get_N_samples(N+budget//100)

    for i in range(len(test_models['times'])):
        time_spent = time_spent + test_models['times'][i]
        if time_spent >= budget:
            break

    X = test_models['X'][:i]
    A = test_models['norm_A'][:i]
    AT = test_models['norm_AT'][:i]
    labels = test_models['val_acc'][:i]
    test_labels = test_models['test_acc'][:i]

    pred_acc = reg.predict(np.concatenate([X, A], axis=-1)).ravel()

    selected_val = labels
    selected_test = test_labels

    best_val_idx = np.argmax(selected_val)

    val_acc.append(selected_val[best_val_idx])
    test_acc.append(selected_test[best_val_idx])

    np_val_avg.append(np.max(val_acc))
    np_val_std.append(np.std(val_acc))

    np_test_avg.append(np.max(test_acc))
    np_test_std.append(np.std(test_acc))

  0%|          | 0/368 [00:00<?, ?it/s]

In [10]:
result_cnn = {
    'np_val_avg': np_val_avg,
    'np_test_avg': np_test_avg,
    'np_val_std': np_val_std,
    'np_test_std': np_test_std
}

In [11]:
np.save('outputs/ablation_study_by_arch_time_cnn.npy', result_cnn)

In [12]:
# GCN
N = 172
train_data = get_N_samples(N)
reg = regressor([train_data['X'], train_data['norm_A'], train_data['norm_AT']], train_data['val_acc'])
train_time = np.sum(train_data['times'])

np_val_avg, np_test_avg = [], []
np_val_std, np_test_std = [], []

val_acc, test_acc = [], []
for budget in tqdm(range(int(train_time), int(MAX_TIME_BUDGET), 1600)): # 500 loops
    time_spent = 0
    test_models = get_N_samples(N+budget//100)

    for i in range(len(test_models['times'])):
        time_spent = time_spent + test_models['times'][i]
        if time_spent >= budget:
            break

    X = test_models['X'][:i]
    A = test_models['norm_A'][:i]
    AT = test_models['norm_AT'][:i]
    labels = test_models['val_acc'][:i]
    test_labels = test_models['test_acc'][:i]

    pred_acc = reg.predict([X, A, AT]).ravel()

    selected_val = labels
    selected_test = test_labels

    best_val_idx = np.argmax(selected_val)

    val_acc.append(selected_val[best_val_idx])
    test_acc.append(selected_test[best_val_idx])

    np_val_avg.append(np.max(val_acc))
    np_val_std.append(np.std(val_acc))

    np_test_avg.append(np.max(test_acc))
    np_test_std.append(np.std(test_acc))

  0%|          | 0/358 [00:00<?, ?it/s]

In [13]:
result_gcn = {
    'np_val_avg': np_val_avg,
    'np_test_avg': np_test_avg,
    'np_val_std': np_val_std,
    'np_test_std': np_test_std
}

In [14]:
np.save('outputs/ablation_study_by_arch_time_gcn.npy', result_gcn)